In [1]:
using Statistics
using Plots

using BlackBoxOptim
using BlackBoxOptim: num_func_evals

using MATLAB

In [2]:
sess = MSession()

# Open ADRIA project
adria_loc = "c:/programs/ownCloud/projects/AIMS/ADRIA_repo"
eval_string(sess, "prj_dir = '$(adria_loc)';")
eval_string(sess, "openProject(prj_dir);")

In [3]:
mat"""disp(prj_dir)"""

c:/programs/ownCloud/projects/AIMS/ADRIA_repo


In [4]:
mat"""
ai = ADRIA();
ai = ai.loadConnectivity(strcat(prj_dir, '/Inputs/MooreTPMean.xlsx'));

% Intervention parameter details
intervention_details = ai.interventions;

% Parameter table to update
sample_params = ai.sample_defaults;

% Extract coral parameters used
% For sensitivity analysis, these values would change
% and so would need to be extracted for every run.
% But here we're only perturbing intervention parameters
% so this can remain constant over all runs
[~, ~, coral_params] = ai.splitParameterTable(sample_params);

% Transfer intervention parameter bounds to Julia
$l_bnd = intervention_details.lower_bound;
$u_bnd = intervention_details.upper_bound;

% MATLAB C API limitation with strings. 
% Have to call this function to extract usable character vectors.
$param_names = convertContainedStringsToChars(intervention_details.name);
"""

In [5]:
length(param_names)

10

In [6]:
mean_TC_progress = []
mean_E_progress = []
function callback(oc)
    bs = best_fitness(oc)
    metric_values = bs.orig  # Use `bs.agg` for aggregated values
    push!(mean_TC_progress, metric_values[1])
    push!(mean_E_progress, metric_values[2])

    nf = num_func_evals(oc)

    # Display last 20 results
    plot_window = min(20, length(mean_TC_progress)-1)
    fig = plot(mean_TC_progress[end-plot_window:end], label="TC fitness")
    fig2 = plot(mean_E_progress[end-plot_window:end], label="E fitness")
    d = plot(fig, fig2, size=(1000, 400), layout=(1,2))
    display(d)
end

callback (generic function with 1 method)

In [7]:
"""Helper function to collect the average metric result at end of simulation time."""
function mean_result(Ymet)
    # Reorder so that simulation index is first
    Ymet = permutedims(Ymet, [3, 1, 2])

    # Calculate element-wise mean across simulations
    # i.e., mean of value at same time/site across replicates
    mean_Y = mean(Ymet, dims=1)

    # Drop singleton dimension (equivalent to `squeeze()` in matlab)
    # and get last entry (last time step)
    mean_Y = dropdims(mean_Y, dims=1)[end, :];

    # Average value at end of simulation time
    return mean(mean_Y)
end

mean_result

In [8]:
function objfunc(X)

    # Update parameter values by name
    for (pn, val) in zip(param_names, X)
        eval_string(sess, "sample_params.('$pn') = $val;")
    end

    mat"""
    % Run simulation with updated sample values
    Y = ai.run(sample_params, sampled_values=true, nreps=50);

    % Collate metrics
    $Ymet = collectMetrics(Y, coral_params, {@coralTaxaCover, @coralEvenness});
    """

    # Get total coral cover over time/sites for each simulation
    TC = Ymet["coralTaxaCover"]["total_cover"]
    mean_TC = mean_result(TC)
    mean_evenness = mean_result(Ymet["coralEvenness"])  # coral evenness

    # Return average results at end of simulation time
    return (mean_TC, mean_evenness)
end

objfunc (generic function with 1 method)

In [9]:
# Ignore the multi-objective GA option for MCDA
# Constains to rank order or TOPSIS
u_bnd[2] = 2.9

2.9

In [10]:
n_params = length(param_names)

res = bboptimize(objfunc; Method=:borg_moea,
                 FitnessScheme=ParetoFitnessScheme{2}(is_minimizing=false),
                 SearchRange=collect(zip(l_bnd, u_bnd)),
                 ϵ=0.05,
                 NumDimensions=n_params, MaxTime=3600*4,
                 TraceInterval=600, PopulationSize=500);  # CallbackFunction=callback, CallbackInterval=900.0, TraceMode=:silent

Starting parallel pool (parpool) using the 'local' profile ...
Connected to the parallel pool (number of workers: 4).
Starting optimization with optimizer BlackBoxOptim.BorgMOEA{EpsBoxDominanceFitnessScheme{2, Float64, false, typeof(sum)}, BlackBoxOptim.ProblemEvaluator{Tuple{Float64, Float64}, IndexedTupleFitness{2, Float64}, EpsBoxArchive{2, Float64, EpsBoxDominanceFitnessScheme{2, Float64, false, typeof(sum)}}, FunctionBasedProblem{typeof(objfunc), ParetoFitnessScheme{2, Float64, false, typeof(sum)}, ContinuousRectSearchSpace, Nothing}}, FitPopulation{IndexedTupleFitness{2, Float64}}, FixedGeneticOperatorsMixture, RandomBound{ContinuousRectSearchSpace}}
0.00 secs, 0 evals, 0 steps
pop.size=500 arch.size=0 n.restarts=0
1261.17 secs, 501 evals, 1 steps, fitness=(29.39852, 0.59781) agg=29.99633
pop.size=500 arch.size=7 n.restarts=0
1863.23 secs, 748 evals, 205 steps, fitness=(35.58962, 0.60561) agg=36.19523
pop.size=500 arch.size=7 n.restarts=0
2463.35 secs, 918 evals, 346 steps, fitne

In [ ]:
# Before improvements, MaxTime: 4 hours
# Best candidate found: [2.77113, 2.92069, 2.78518, 9.98221e5, 9.99982e5, 1.99449, 11.9998, 0.0827281, 6.34444, 6.99785]
# Fitness: (38.63666, 0.50006) agg=39.13672

# close(sess)